In [1]:
import polars as pl
import sys
import os
import json

sys.path.append(os.path.dirname(os.path.abspath("")))

from libraries.client_stashapp import get_stashapp_client, StashAppClient
from libraries import browser

stash = get_stashapp_client()
stash_client = StashAppClient()

print("🎯 SIMPLIFIED FACIAL/CUMSHOT TAG TEST")
print("=" * 50)
print("✅ Ready to test browser data injection with real tag scenario")

dUsing stash (v0.28.1-89-g642b0f22) endpoint at http://localhost:6969/graphql
dUsing stash (v0.28.1-89-g642b0f22) endpoint at http://localhost:6969/graphql


🎯 SIMPLIFIED FACIAL/CUMSHOT TAG TEST
✅ Ready to test browser data injection with real tag scenario


In [2]:
# Find scenes with Facial tag but no Cumshot tag
# Get the Facial and Cumshot tags using stash_client
stash_tags = stash_client.get_tags_by_names(["Facial", "Cumshot"])

scenes_with_facial_wo_cumshot = stash.find_scenes(
    {"tags": {"value": [stash_tags.facial["id"]], "modifier": "INCLUDES", "excludes": [stash_tags.cumshot["id"]]}},
    fragment="id title date tags { id name }",
)

print(f"\nFound {len(scenes_with_facial_wo_cumshot)} scenes with Facial but no Cumshot tag")


🏷️  Tag Lookup Results:
✅ facial                         -> Facial (ID: 5814)
✅ cumshot                        -> Cumshot (ID: 5626)
Found 2 out of 2 tags

Found 2436 scenes with Facial but no Cumshot tag


In [3]:
# Generate the fix data structure
def generate_cumshot_fix_data():
    """Generate fix data for scenes missing Cumshot tags"""
    
    if not scenes_with_facial_wo_cumshot:
        return []
    
    # Create the fix data structure
    fix_data = [{
        "name": "Missing Cumshot Fix",
        "addTags": [{
            "id": stash_tags.cumshot["id"],
            "label": stash_tags.cumshot["name"]
        }],
        "removeTags": []
    }]
    
    return fix_data

# Generate the fix data
fix_data = generate_cumshot_fix_data()

print("Generated fix data:")
print(json.dumps(fix_data, indent=2))

print(f"\nThis fix will add the Cumshot tag (ID: {stash_tags.cumshot['id']}) to scenes that have Facial but are missing Cumshot")


Generated fix data:
[
  {
    "name": "Missing Cumshot Fix",
    "addTags": [
      {
        "id": "5626",
        "label": "Cumshot"
      }
    ],
    "removeTags": []
  }
]

This fix will add the Cumshot tag (ID: 5626) to scenes that have Facial but are missing Cumshot


In [4]:
# Browser functionality for injecting fix data
def check_browser_connection():
    """Check if Chrome is running with remote debugging enabled"""
    try:
        import requests
        response = requests.get('http://localhost:9222/json', timeout=5)
        if response.status_code == 200:
            pages = json.loads(response.text)
            print(f"✅ Chrome is running with remote debugging")
            print(f"📊 Found {len(pages)} open tabs/pages")
            return True
        else:
            print(f"❌ Chrome debugging port responded with status {response.status_code}")
            return False
    except Exception as e:
        print("❌ Cannot connect to Chrome debugging port")
        print("🔧 Please start Chrome with: --remote-debugging-port=9222")
        return False

def create_scenes_query_url(base_url="https://stash.chiefsclub.com"):
    """Create a Stash query URL for scenes with Facial tag but excluding Cumshot tag"""
    import urllib.parse
    
    # Build the query object for tags
    query = {
        "type": "tags",
        "modifier": "INCLUDES_ALL", 
        "value": {
            "items": [{"id": "5814", "label": "Facial"}],
            "excluded": [{"id": "5626", "label": "Cumshot"}],
            "depth": 0
        }
    }
    
    # Convert to JSON string and use the specific URL encoding format that Stash expects
    query_json = json.dumps(query)
    # Replace curly braces with parentheses and square brackets with %5B %5D
    query_str = query_json.replace("{", "(").replace("}", ")").replace("[", "%5B").replace("]", "%5D")
    # URL encode the query string
    query_str = urllib.parse.quote(query_str, safe="():%5B%5D")
    
    # Build the full URL
    url = f"{base_url}/scenes?c={query_str}&sortby=path&perPage=48&disp=2"
    
    return url

# Check browser connection
browser_ready = check_browser_connection()

✅ Chrome is running with remote debugging
📊 Found 4 open tabs/pages


In [5]:
# Open browser with fix data
def open_scenes_with_fix_data():
    """Open the problematic scenes in browser with fix data injected"""
    
    if not browser_ready:
        print("⚠️  Browser not ready. Start Chrome with:")
        print('Start-Process "chrome.exe" -ArgumentList "--remote-debugging-port=9222", "--user-data-dir=C:\\\\temp\\\\chrome-debug"')
        return False
    
    if not scenes_with_facial_wo_cumshot:
        print("⚠️  No scenes found that need fixing")
        return False
    
    # Create URL for the scenes
    url = create_scenes_query_url()
    
    print(f"🚀 Opening browser with {len(scenes_with_facial_wo_cumshot)} scenes that need Cumshot tags...")
    print(f"📋 Fix data will be injected as window.stashFixesData")
    print(f"🔗 URL: {url}")
    
    # Open the browser tab with fix data
    results = browser.open_or_update_tabs_with_data([url], fix_data, "stashFixesData")
    
    success = results.get(url, False)
    if success:
        print("✅ Successfully opened browser tab with fix data!")
        print("\n💡 In browser console, you can access:")
        print("   console.log(window.stashFixesData)")
        print("   console.log(window.stashFixesData[0].addTags)")
        print(f"\n🏷️  Fix data contains Cumshot tag: ID={stash_tags.cumshot['id']}, Name='{stash_tags.cumshot['name']}'")
    else:
        print("❌ Failed to open browser tab")
    
    return success

# Test the browser functionality
if scenes_with_facial_wo_cumshot:
    print(f"\n🎯 READY TO TEST:")
    print(f"Found {len(scenes_with_facial_wo_cumshot)} scenes with Facial but no Cumshot")
    print(f"Fix data ready: {json.dumps(fix_data, indent=2)}")
    print(f"\n📋 To open browser with fix data, run:")
    print(f"open_scenes_with_fix_data()")
else:
    print("\n✅ No scenes found that need fixing (all scenes with Facial already have Cumshot)")



🎯 READY TO TEST:
Found 2436 scenes with Facial but no Cumshot
Fix data ready: [
  {
    "name": "Missing Cumshot Fix",
    "addTags": [
      {
        "id": "5626",
        "label": "Cumshot"
      }
    ],
    "removeTags": []
  }
]

📋 To open browser with fix data, run:
open_scenes_with_fix_data()


In [6]:
# Test the fixed URL generation
test_url = create_scenes_query_url()
print(f"Generated URL: {test_url}")
print("This should now use tags filtering instead of scene_ids filtering")

Generated URL: https://stash.chiefsclub.com/scenes?c=(%22type%22:%20%22tags%22%2C%20%22modifier%22:%20%22INCLUDES_ALL%22%2C%20%22value%22:%20(%22items%22:%20%5B(%22id%22:%20%225814%22%2C%20%22label%22:%20%22Facial%22)%5D%2C%20%22excluded%22:%20%5B(%22id%22:%20%225626%22%2C%20%22label%22:%20%22Cumshot%22)%5D%2C%20%22depth%22:%200))&sortby=path&perPage=48&disp=2
This should now use tags filtering instead of scene_ids filtering


In [7]:
# If the URL looks correct, run the full function
print("If the URL above looks correct, run the full function:")
print("open_scenes_with_fix_data()")
print("\nOr run it directly:")
open_scenes_with_fix_data()


If the URL above looks correct, run the full function:
open_scenes_with_fix_data()

Or run it directly:
🚀 Opening browser with 2436 scenes that need Cumshot tags...
📋 Fix data will be injected as window.stashFixesData
🔗 URL: https://stash.chiefsclub.com/scenes?c=(%22type%22:%20%22tags%22%2C%20%22modifier%22:%20%22INCLUDES_ALL%22%2C%20%22value%22:%20(%22items%22:%20%5B(%22id%22:%20%225814%22%2C%20%22label%22:%20%22Facial%22)%5D%2C%20%22excluded%22:%20%5B(%22id%22:%20%225626%22%2C%20%22label%22:%20%22Cumshot%22)%5D%2C%20%22depth%22:%200))&sortby=path&perPage=48&disp=2
Got response from Chrome: 200
Found 4 existing pages
Creating new tab for https://stash.chiefsclub.com/scenes?c=(%22type%22:%20%22tags%22%2C%20%22modifier%22:%20%22INCLUDES_ALL%22%2C%20%22value%22:%20(%22items%22:%20%5B(%22id%22:%20%225814%22%2C%20%22label%22:%20%22Facial%22)%5D%2C%20%22excluded%22:%20%5B(%22id%22:%20%225626%22%2C%20%22label%22:%20%22Cumshot%22)%5D%2C%20%22depth%22:%200))&sortby=path&perPage=48&disp=2
Injec

True